In [1]:
pip install torch torch-geometric scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader


In [9]:
class GCN_LSTM(nn.Module):
    def __init__(self, in_channels, hidden_dim, lstm_hidden, num_classes):
        super(GCN_LSTM, self).__init__()
        self.gcn1 = GCNConv(in_channels, hidden_dim)
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, lstm_hidden, batch_first=True)
        self.fc = nn.Linear(lstm_hidden, num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))
        # Reshape for LSTM without adding a batch dimension
        x = x.view(x.size(0), 1, -1) # Reshape to (num_nodes, seq_len, features)
        lstm_out, _ = self.lstm(x)
        # Use the last output for each node
        x = self.fc(lstm_out[:, -1, :])
        return F.log_softmax(x, dim=1)

In [10]:
# Create a Sample FANET Graph Data
def create_fanet_data():
    node_features = torch.rand((10, 5))  # 10 nodes, 5 features each
    edge_index = torch.tensor([
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 3, 4, 5],  # Source nodes
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 3, 4, 5, 6]   # Target nodes
    ], dtype=torch.long)  # Edges
    labels = torch.randint(0, 2, (10,))  # Random binary classification labels

    return Data(x=node_features, edge_index=edge_index, y=labels)


In [11]:
# Train Function
def train(model, data, optimizer, criterion, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

In [12]:
# Evaluate Function
def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        accuracy = (pred == data.y).sum().item() / data.y.size(0)
    print(f'Accuracy: {accuracy:.4f}')

In [13]:
# Initialize Model, Optimizer, and Loss Function
data = create_fanet_data()
model = GCN_LSTM(in_channels=5, hidden_dim=16, lstm_hidden=8, num_classes=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()


In [14]:
# Train and Evaluate
train(model, data, optimizer, criterion, epochs=100)
evaluate(model, data)


Epoch 0, Loss: 0.6825
Epoch 10, Loss: 0.6555
Epoch 20, Loss: 0.5654
Epoch 30, Loss: 0.3496
Epoch 40, Loss: 0.1979
Epoch 50, Loss: 0.0700
Epoch 60, Loss: 0.0168
Epoch 70, Loss: 0.0061
Epoch 80, Loss: 0.0033
Epoch 90, Loss: 0.0022
Accuracy: 1.0000
